# Introdução

O presente notebook é efetuado no âmbito do desafio da 3ª edição do #PlottingGoodDSSG promovido por `Data Science for Social Good Portugal` e a elaboração do mesmo teve como propósito a aquisição de skills nas bibliotecas geopandas, folium e outras.

Tentei replicar o notebook disponibilizado pela `dssg pt` e plotar alguns mapas, com especial foco no distrito de faro e criação os ficheiros html dos mapas.

**Créditos:** dssg pt https://github.com/dssgPT/Plotting-Good-DSSG/tree/main/desafios/003_Liberdade_OSM

## 1. Importação de bibliotecas e definição de funções

In [5]:
import geopandas as gpd
import pygeos as pygeos

from geopy import distance
from sklearn.preprocessing import LabelEncoder

import pandas as pd
from simpledbf import Dbf5

import folium

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [6]:
def calc_dist(df):
    x, y = df['centroid_x'], df['centroid_y']

    x_nn, y_nn = df['nn_centroid_x'], df['nn_centroid_y']

    return distance.distance((y, x), (y_nn, x_nn)).meters


## 2. Ruas alusivas ao 25 de aril

### 2.1 Carregamento do GeoDataFrame

In [7]:
roads_shp = "../dados_osm/portugal-latest-free.shp/gis_osm_roads_free_1.shp"


In [8]:
roads = gpd.read_file(roads_shp)

In [9]:
#roads.head()

### 2.2 Expressões a pesquisar

In [10]:
freedom_roads = roads[(roads.name.str.contains("25 de Abril")) | (roads.name.str.contains("Liberdade")) | (
    roads.name.str.contains("Salgueiro Maia")) | (roads.name.str.contains("Capitães de Abril"))]
#freedom_roads

### 2.3 Drop e renomeação de colunas

In [11]:
df_roads = freedom_roads[['osm_id', 'name', 'geometry']]
df_roads = df_roads.rename(columns={'name': 'rua'})
df_roads.head(2)


,osm_id,rua,geometry
224,4453413,Avenida da Liberdade,"LINESTRING (-9.02960 39.75781, -9.02954 39.757..."
237,4472168,Avenida da Liberdade,"LINESTRING (-9.03221 39.75903, -9.03204 39.758..."


### 2.4 Visualização das ruas no mapa

Cria ficheiro Geojson para plotar no folium

In [12]:
geojson_roads = df_roads.to_crs(epsg='4326').to_json()


In [13]:

mapa = folium.Map([37.125756, -7.649822],
                  zoom_start=8,
                  tiles='cartodbpositron')

points = folium.features.GeoJson(geojson_roads)

mapa.add_child(points)
mapa.save('../outputs/ruas_portugal.html')          # Ao criar na mesma celula o ficheiro html o mapa não é renderizado no IDE e o ficheiro .ipynb torna-se mais pequeno

### 2.5 Criação de ficheiro html

In [14]:
mapa.save('../outputs/ruas_portugal.html')

## 3. Concelhos e ruas

### 3.1 Carregamento do GeoDataFrame dos concelhos

In [15]:
concelhos = gpd.read_file('../dados_osm/concelhos-shapefile/concelhos.shp',
                          SHAPE_RESTORE_SHX='YES')

In [16]:
concelhos.head()

,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,HASC_2,CCN_2,CCA_2,TYPE_2,ENGTYPE_2,NL_NAME_2,VARNAME_2,geometry
0,182,PRT,Portugal,1,Évora,1,Évora,PT.EV.EV,0,0705,Concelho,Municipality,None,None,"POLYGON ((-7.79291 38.76507, -7.79287 38.76506..."
1,182,PRT,Portugal,1,Évora,2,Alandroal,PT.EV.AL,0,0701,Concelho,Municipality,None,None,"POLYGON ((-7.25937 38.77351, -7.25921 38.77343..."
2,182,PRT,Portugal,1,Évora,3,Arraiolos,PT.EV.AR,0,0702,Concelho,Municipality,None,None,"POLYGON ((-7.88611 38.92495, -7.88580 38.92472..."
3,182,PRT,Portugal,1,Évora,4,Borba,PT.EV.BO,0,0703,Concelho,Municipality,None,None,"POLYGON ((-7.46362 38.92344, -7.46344 38.92329..."
4,182,PRT,Portugal,1,Évora,5,Estremoz,PT.EV.ES,0,0704,Concelho,Municipality,None,None,"POLYGON ((-7.52770 39.00080, -7.52765 39.00066..."


### 3.2 Concelhos do distrito de Faro

In [17]:
concelhos.loc[concelhos.NAME_1 =='Faro']

,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,HASC_2,CCN_2,CCA_2,TYPE_2,ENGTYPE_2,NL_NAME_2,VARNAME_2,geometry
120,182,PRT,Portugal,9,Faro,120,Albufeira,PT.FA.AB,0,0801,Concelho,Municipality,None,None,"MULTIPOLYGON (((-8.28744 37.07175, -8.28748 37..."
121,182,PRT,Portugal,9,Faro,121,Alcoutim,PT.FA.AC,0,0802,Concelho,Municipality,None,None,"POLYGON ((-7.56960 37.52726, -7.56915 37.52715..."
122,182,PRT,Portugal,9,Faro,121,Alcoutim,PT.FA.AC,0,0814,Concelho,Municipality,None,None,"MULTIPOLYGON (((-7.61847 37.11625, -7.61847 37..."
123,182,PRT,Portugal,9,Faro,122,Aljezur,PT.FA.AJ,0,0803,Concelho,Municipality,None,None,"MULTIPOLYGON (((-8.90731 37.16032, -8.90735 37..."
124,182,PRT,Portugal,9,Faro,123,Castro Marim,PT.FA.CM,0,0804,Concelho,Municipality,None,None,"POLYGON ((-7.56433 37.39458, -7.56424 37.39458..."
125,182,PRT,Portugal,9,Faro,124,Faro,PT.FA.FA,0,0805,Concelho,Municipality,None,None,"MULTIPOLYGON (((-7.88042 36.97347, -7.88042 36..."
126,182,PRT,Portugal,9,Faro,125,Lagoa,PT.FA.LA,0,0806,Concelho,Municipality,None,None,"MULTIPOLYGON (((-8.44033 37.08669, -8.44038 37..."
127,182,PRT,Portugal,9,Faro,126,Lagos,PT.FA.LS,0,0807,Concelho,Municipality,None,None,"MULTIPOLYGON (((-8.66711 37.07995, -8.66716 37..."
128,182,PRT,Portugal,9,Faro,127,Loulé,PT.FA.LO,0,0808,Concelho,Municipality,None,None,"POLYGON ((-7.89775 37.41819, -7.89752 37.41811..."
129,182,PRT,Portugal,9,Faro,128,Monchique,PT.FA.MO,0,0809,Concelho,Municipality,None,None,"POLYGON ((-8.56828 37.41016, -8.56753 37.40986..."


Por simples observação verifica-se que o concelho de ``Alcoutim`` (código ``ID_2`` de __121__) possui 2 registos. Correspondendo um ao código de concelho `CCA_2` de __0802__ e outro a __0814__.

Vejamos se esta incoerência verifica-se noutros concelhos.

In [18]:
concelhos.groupby(['NAME_2'])['NAME_2'].count(
).sort_values(ascending=False).head()


NAME_2
Braga       2
Lagoa       2
Calheta     2
Alcoutim    2
Abrantes    1
Name: NAME_2, dtype: int64

In [19]:
concelhos.loc[concelhos.NAME_2.isin(
    ['Braga', 'Lagoa', 'Calheta', 'Alcoutim'])].sort_values('NAME_2')


,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,HASC_2,CCN_2,CCA_2,TYPE_2,ENGTYPE_2,NL_NAME_2,VARNAME_2,geometry
121,182,PRT,Portugal,9,Faro,121,Alcoutim,PT.FA.AC,0,0802,Concelho,Municipality,None,None,"POLYGON ((-7.56960 37.52726, -7.56915 37.52715..."
122,182,PRT,Portugal,9,Faro,121,Alcoutim,PT.FA.AC,0,0814,Concelho,Municipality,None,None,"MULTIPOLYGON (((-7.61847 37.11625, -7.61847 37..."
68,182,PRT,Portugal,5,Braga,69,Braga,PT.BR.BR,0,0303,Concelho,Municipality,None,None,"POLYGON ((-8.35704 41.61721, -8.35661 41.61719..."
69,182,PRT,Portugal,5,Braga,69,Braga,PT.BR.BR,0,0308,Concelho,Municipality,None,None,"POLYGON ((-8.31783 41.54539, -8.31739 41.54216..."
34,182,PRT,Portugal,3,Azores,35,Calheta,PT.AC.CA,0,4501,Concelho,Municipality,None,None,"MULTIPOLYGON (((-28.03560 38.60697, -28.03561 ..."
183,182,PRT,Portugal,13,Madeira,182,Calheta,PT.MA.CT,0,3101,Concelho,Municipality,None,None,"MULTIPOLYGON (((-17.24440 32.78358, -17.24445 ..."
37,182,PRT,Portugal,3,Azores,38,Lagoa,PT.AC.LG,0,4201,Concelho,Municipality,None,None,"POLYGON ((-25.55903 37.77534, -25.55883 37.775..."
126,182,PRT,Portugal,9,Faro,125,Lagoa,PT.FA.LA,0,0806,Concelho,Municipality,None,None,"MULTIPOLYGON (((-8.44033 37.08669, -8.44038 37..."


Conforme se observa a situação ocorre também nos concelhos de `Braga`, `Lagoa` e `Calheta`.
Por consulta aos códigos dos concelhos em https://info.portaldasfinancas.gov.pt/pt/apoio_contribuinte/Documents/ListaFreguesiasVigentes_SFs.xls, por concatenação dos campos `Código Distrito` e `Código Concelho`, conclui-se que:

    - 0814 - corresponde a Tavira
    - 0308 - corresponde a Guimarães

Relativamente a `Calheta` não se verifica anomalia pois existe concelhos com o mesmo nome nos Açores e na Madeira, o mesmo acontece com `Lagoa`, pois também temos concelhos com o mesmo nome nos Açores e no distrito de Faro.

Vamos corrigir o dataset

In [20]:
concelhos.loc[concelhos.CCA_2 == '0814', 'NAME_2'] = 'Tavira'
concelhos.loc[concelhos.CCA_2 == '0308', 'NAME_2'] = 'Guimarães'

### 3.3 Drop e renomeação de colunas

In [21]:
df_concelhos = concelhos[['CCA_2', 'NAME_1','NAME_2', 'geometry']]
df_concelhos = df_concelhos.rename(
    columns={'NAME_1': 'distrito','NAME_2': 'concelho'})
df_concelhos


,CCA_2,distrito,concelho,geometry
0,0705,Évora,Évora,"POLYGON ((-7.79291 38.76507, -7.79287 38.76506..."
1,0701,Évora,Alandroal,"POLYGON ((-7.25937 38.77351, -7.25921 38.77343..."
2,0702,Évora,Arraiolos,"POLYGON ((-7.88611 38.92495, -7.88580 38.92472..."
3,0703,Évora,Borba,"POLYGON ((-7.46362 38.92344, -7.46344 38.92329..."
4,0704,Évora,Estremoz,"POLYGON ((-7.52770 39.00080, -7.52765 39.00066..."
...,...,...,...,...
303,1820,Viseu,Tarouca,"POLYGON ((-7.72591 41.08079, -7.72442 41.08028..."
304,1821,Viseu,Tondela,"POLYGON ((-8.12772 40.64028, -8.12751 40.64027..."
305,1822,Viseu,Vila Nova de Paiva,"POLYGON ((-7.77933 40.93439, -7.77835 40.93436..."
306,1823,Viseu,Viseu,"POLYGON ((-7.80686 40.83789, -7.80653 40.83787..."


### 3.4 Visualização dos concelhos no mapa

In [22]:
geojson_concelhos = df_concelhos.to_crs(epsg='4326').to_json()


In [23]:
mapa_concelho = folium.Map([37.125756, -7.649822],
                           zoom_start=8,
                           tiles='cartodbpositron')

points_ruas = folium.features.GeoJson(geojson_roads)
points_concelhos = folium.features.GeoJson(geojson_concelhos)

mapa_concelho.add_child(points_ruas)
mapa_concelho.add_child(points_concelhos)
mapa_concelho.save('../outputs/ruas_e_concelhos_portugal.html') # Ao criar na mesma celula o ficheiro html o mapa não é renderizado no IDE e o ficheiro .ipynb torna-se mais pequeno

### 3.5 Criação de ficheiro html

In [24]:
mapa_concelho.save('../outputs/ruas_e_concelhos_portugal.html')


## 4. Algarve

### 4.1 Merge `df_roads` e `df_concelhos`

In [21]:
df_roads_merged = df_roads.copy()
df_roads_merged['centroid_json'] = df_roads_merged.geometry.apply(lambda x: x.centroid)
df_roads_merged = df_roads_merged.sjoin(df_concelhos[['distrito','concelho', 'geometry']], how="left")
df_roads_merged = df_roads_merged[[
    'osm_id', 'rua', 'concelho', 'distrito', 'geometry', 'centroid_json']]


In [1]:
df_roads_merged.head(2)


NameError: name 'df_roads_merged' is not defined

### 4.2 Ruas e concelhos do Algarve

#### 4.2.1 Ruas do Algarve

In [23]:
df_ruas_algarve = df_roads_merged.loc[df_roads_merged.distrito == 'Faro'].drop(columns=
    'centroid_json')
df_ruas_algarve.head()


,osm_id,rua,concelho,distrito,geometry
1951,18506519,Rua 25 de Abril,Lagos,Faro,"LINESTRING (-8.73729 37.08695, -8.73722 37.086..."
6742,24005470,Rua 25 de Abril,Aljezur,Faro,"LINESTRING (-8.80039 37.30348, -8.80075 37.303..."
7478,24717924,Rua da Liberdade,Lagos,Faro,"LINESTRING (-8.65473 37.14843, -8.65481 37.148..."
7689,24824628,Largo da Liberdade,Lagos,Faro,"LINESTRING (-8.65546 37.14909, -8.65549 37.149..."
7884,24841817,Rua 25 de Abril 1974,Lagos,Faro,"LINESTRING (-8.67211 37.10167, -8.67190 37.101..."


#### 4.2.2 Concelhos do Algarve

In [2]:
df_concelhos_algarve = df_concelhos.loc[df_concelhos.distrito == 'Faro']
df_concelhos_algarve


NameError: name 'df_concelhos' is not defined

### 4.3 Visualização ruas por concelho do Algarve

In [25]:
geojson_ruas_algarve = df_ruas_algarve.to_crs(epsg='4326').to_json()
geojson_conc_algarve = df_concelhos_algarve.to_crs(epsg='4326').to_json()

In [26]:
x = df_concelhos_algarve.centroid.x.mean()
y = df_concelhos_algarve.centroid.y.mean()

C:\Users\ernes\AppData\Local\Temp\ipykernel_3100\3584048611.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  x = df_concelhos_algarve.centroid.x.mean()
C:\Users\ernes\AppData\Local\Temp\ipykernel_3100\3584048611.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  y = df_concelhos_algarve.centroid.y.mean()


In [27]:
mapa_algarve = folium.Map([y, x],
                           zoom_start=9,
                           tiles='cartodbpositron')

points_ruas = folium.features.GeoJson(geojson_ruas_algarve)
points_concelhos = folium.features.GeoJson(geojson_conc_algarve)

mapa_algarve.add_child(points_ruas)
mapa_algarve.add_child(points_concelhos)

mapa_algarve.save('../outputs/ruas_e_concelhos_algarve.html') # Ao criar o ficheiro html o mapa não é renderizado no IDE e o .ipynb fica mais pequeno


### 4.4 Criação de ficheiro html

In [28]:
mapa_algarve.save('../outputs/ruas_e_concelhos_algarve.html')


## 5. Ruas por concelho (Algarve)

### 5.1 Carregamento do GeoDataFrame das Freguesias

In [29]:
freguesias = gpd.read_file(
    '../dados_osm/Cont_AAD_CAOP2021/Cont_AAD_CAOP2021.shp', SHAPE_RESTORE_SHX='YES')


In [30]:
freguesias.loc[freguesias.Concelho=='Tavira']


,Dicofre,Freguesia,Concelho,Distrito,TAA,Area_T_ha,Area_EA_ha,Des_Simpli,geometry
117,081408,Santa Luzia,Tavira,Faro,ÁREA PRINCIPAL,850.11,850.11,Santa Luzia,"POLYGON ((44124.924 -284909.918, 44001.315 -28..."
147,081411,União das freguesias de Luz de Tavira e Santo ...,Tavira,Faro,ÁREA PRINCIPAL,5991.23,5991.23,Luz de Tavira e Santo Estêvão,"POLYGON ((39830.680 -285449.774, 39857.751 -28..."
266,081404,Santa Catarina da Fonte do Bispo,Tavira,Faro,ÁREA PRINCIPAL,11758.82,11758.82,Santa Catarina da Fonte do Bispo,"POLYGON ((36801.125 -267518.604, 36788.657 -26..."
268,081410,União das freguesias de Conceição e Cabanas de...,Tavira,Faro,ÁREA PRINCIPAL,6944.44,6944.44,Conceição e Cabanas de Tavira,"POLYGON ((47725.490 -266319.773, 47726.940 -26..."
277,081412,União das freguesias de Tavira (Santa Maria e ...,Tavira,Faro,ÁREA PRINCIPAL,14799.45,14799.45,Tavira (Santa Maria e Santiago),"POLYGON ((46705.715 -266056.914, 46701.996 -26..."
344,081401,Cachopo,Tavira,Faro,ÁREA PRINCIPAL,20352.65,20352.65,Cachopo,"POLYGON ((31092.759 -253628.910, 31092.629 -25..."


### 5.2 Merge das ruas do algarve com as freguesias

In [31]:
freguesias = freguesias.to_crs(epsg='4326')


In [32]:
df_ruas_algarve_merged = df_ruas_algarve.sjoin(
    freguesias[['Freguesia', 'geometry']])
df_ruas_algarve_merged.sample(3)


,osm_id,rua,concelho,distrito,geometry,index_right,Freguesia
213624,157157185,Rua da Liberdade,Aljezur,Faro,"LINESTRING (-8.77276 37.42876, -8.77276 37.428...",379,Odeceixe
397477,252328072,Avenida 25 de Abril,Albufeira,Faro,"LINESTRING (-8.25195 37.12868, -8.25360 37.129...",129,Ferreiras
804174,622463659,Rua 25 de Abril,Olhão,Faro,"LINESTRING (-7.87252 37.05574, -7.87252 37.055...",35,Pechão


In [33]:
len(df_ruas_algarve_merged)


285

### 5.3 Drop registos em duplicado

In [34]:
ruas_unicas_freguesia = df_ruas_algarve_merged.drop_duplicates(
    ['rua', 'concelho', 'Freguesia'])
ruas_unicas_freguesia

,osm_id,rua,concelho,distrito,geometry,index_right,Freguesia
1951,18506519,Rua 25 de Abril,Lagos,Faro,"LINESTRING (-8.73729 37.08695, -8.73722 37.086...",120,Luz
464633,300140567,Travessa da Liberdade,Lagos,Faro,"LINESTRING (-8.73694 37.10275, -8.73684 37.102...",120,Luz
464732,300140671,Rua Bairro da Liberdade,Lagos,Faro,"LINESTRING (-8.73781 37.10241, -8.73740 37.102...",120,Luz
6742,24005470,Rua 25 de Abril,Aljezur,Faro,"LINESTRING (-8.80039 37.30348, -8.80075 37.303...",342,Aljezur
11655,27739410,Largo da Liberdade,Aljezur,Faro,"LINESTRING (-8.80361 37.31654, -8.80358 37.316...",342,Aljezur
...,...,...,...,...,...,...,...
395641,251153341,Largo da Liberdade,Aljezur,Faro,"LINESTRING (-8.86127 37.19569, -8.86125 37.195...",264,Bordeira
624706,416250980,Rua 25 de Abril,Alcoutim,Faro,"LINESTRING (-7.47136 37.47201, -7.47137 37.471...",383,União das freguesias de Alcoutim e Pereiro
684614,462739812,Rua 25 de Abril,Vila do Bispo,Faro,"LINESTRING (-8.90967 37.08204, -8.90905 37.08202)",134,Vila do Bispo e Raposeira
806536,625139868,Largo 25 de Abril,Olhão,Faro,"LINESTRING (-7.82152 37.05751, -7.82155 37.057...",55,Quelfes


### 5.4 Qtd de ruas por concelho

In [3]:
for index, concelho in df_concelhos_algarve.iterrows():
    
    qtd_ruas = len(ruas_unicas_freguesia[ruas_unicas_freguesia.intersects(
        concelho.geometry)])

    df_concelhos_algarve.loc[index, 'ruas'] = qtd_ruas


NameError: name 'df_concelhos_algarve' is not defined

### 5.5 Carregamento da qtd ruas no mapa

In [36]:
for _, concelho in df_concelhos_algarve.iterrows():

    concelho_geojson = folium.features.GeoJson(concelho.geometry,
    style_function = lambda feature: {'color': 'blue', 'weight': 2,
                                      'fillOpacity': 0.1})

    popup = folium.Popup("""
    {}<br> {} ruas""".format(concelho.concelho, str(int(concelho.ruas))), max_width=300)
    popup.add_to(concelho_geojson)

    concelho_geojson.add_to(mapa_algarve)


### 5.6 Visualização do mapa e criação de ficheiro html

In [40]:
#mapa_algarve  # Renderiza o mapa no IDE (mas torna o notebook mais pesado)

In [38]:
mapa_algarve.save('../outputs/algarve_total.html')